In [3]:
import numpy as np
from itertools import combinations
import random

# Constants
BOX_WIDTH = 100
BOX_HEIGHT = 100
NUM_PARTICLES = 3
PARTICLE_RADII = {'A': 1, 'B': 1, 'C': 8}  # Define radii for each species
INITIAL_SPEEDS = {'A': 4, 'B': 4, 'C': 4}
NUM_SIMULATIONS = 1000

# Particle class
class Particle:
    def __init__(self, species, x, y, vx, vy, radius):
        self.species = species
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.radius = radius  # Add radius attribute

# Function to update positions
def update_positions(particles):
    for particle in particles:
        particle.x += particle.vx
        particle.y += particle.vy
        # Wall collisions
        if particle.x > BOX_WIDTH - particle.radius or particle.x < particle.radius:
            particle.vx *= -1
        if particle.y > BOX_HEIGHT - particle.radius or particle.y < particle.radius:
            particle.vy *= -1

def handle_collisions(particles):
    for p1, p2 in combinations(particles, 2):
        if np.hypot(p1.x - p2.x, p1.y - p2.y) < (p1.radius + p2.radius):
            # Determine the new species and update both the species and radius accordingly
            new_species = None
            if {p1.species, p2.species} == {'A', 'B'}:
                new_species = 'C'
            elif {p1.species, p2.species} == {'A', 'C'}:
                new_species = 'B'
            elif {p1.species, p2.species} == {'B', 'C'}:
                new_species = 'A'

            if new_species:
                # Update species
                p1.species = new_species
                p2.species = new_species
                # Update radii to match the new species
                p1.radius = PARTICLE_RADII[new_species]
                p2.radius = PARTICLE_RADII[new_species]

# Function to run a single simulation
def run_simulation():
    # Initialize particles with random angles, speeds, and radii
    particles = []
    for species in ['A', 'B', 'C']:
        radius = PARTICLE_RADII[species]  # Get radius based on species
        for _ in range(NUM_PARTICLES):
            angle = random.uniform(0, 2 * np.pi)
            speed = INITIAL_SPEEDS[species]
            v_x = speed * np.cos(angle)
            v_y = speed * np.sin(angle)
            particles.append(Particle(species,
                                      random.uniform(radius, BOX_WIDTH - radius),
                                      random.uniform(radius, BOX_HEIGHT - radius),
                                      v_x, v_y,
                                      radius))  # Pass radius to Particle constructor
    # Simulation loop
    step = 0
    while step < 30000:  # Max steps to prevent infinite loop
        update_positions(particles)
        handle_collisions(particles)
        
        # Check if only one species remains
        remaining_species = set(particle.species for particle in particles)
        if len(remaining_species) == 1:
            break  # End simulation early if only one species remains
        step += 1

    return remaining_species

# Initialize a dictionary to count occurrences of final species
final_species_count = {'A': 0, 'B': 0, 'C': 0}

# Run simulations and update the count for the final species
for _ in range(NUM_SIMULATIONS):
    outcome = run_simulation()
    for species in outcome:
        final_species_count[species] += 1

# Calculate total counts to use in probability calculation
total_counts = sum(final_species_count.values())

# Calculate and print probabilities for each species
print("Probabilities of each species being the final one:")
probabilities = {}
for species, count in final_species_count.items():
    if total_counts > 0:  # Avoid division by zero
        probabilities[species] = count / total_counts
    else:
        probabilities[species] = 0
    print(f"Prob({species}) = {probabilities[species]:.3f}")


Probabilities of each species being the final one:
Prob(A) = 0.529
Prob(B) = 0.471
Prob(C) = 0.000
